# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意（`ORDER BY` なし）
* `NOT IN` 回避（必要なら `EXISTS` / `LEFT JOIN ... IS NULL`）
* 判定は **ID 基準**、表示は仕様どおり（`id, name, sex, salary`）

## 1) 問題

* `Salary.sex` の `'m'` と `'f'` を **単一の UPDATE 文で相互に入れ替える**（一時表・サブクエリ不要）
* 入力: `Salary(id INT PRIMARY KEY, name TEXT, sex ENUM('m','f') または TEXT + CHECK, salary INT)`
* 出力: `Salary` 全行で `sex` が `'m' <-> 'f'` に反転。他列は不変

## 2) 最適解（単一クエリ）

> 今回はウィンドウ不要。**CASE 一発更新**が最短・明快・安全。

```sql
UPDATE Salary
SET sex = CASE sex
            WHEN 'm' THEN 'f'
            WHEN 'f' THEN 'm'
          END;

Runtime 212 ms
Beats 31.19%

```

* `ENUM('m','f')` でも文字列代入で整合します。
* もし将来 `'m'/'f'` 以外を許す可能性があるなら、無関係行の無駄更新を避けるために `WHERE` を付けてもよい（任意）:

```sql
UPDATE Salary
SET sex = CASE sex WHEN 'm' THEN 'f' WHEN 'f' THEN 'm' END
WHERE sex IN ('m','f');

Runtime 193 ms
Beats 65.45%

```

### 代替（等価・短文）

```sql
-- 読みやすさ重視
UPDATE Salary
SET sex = CASE WHEN sex = 'm' THEN 'f' ELSE 'm' END;

Runtime 193 ms
Beats 65.45%

-- 関数を使った対称表現（好みで）
UPDATE Salary
SET sex = (ARRAY['f','m'])[CASE WHEN sex = 'm' THEN 1 ELSE 2 END];

Runtime 191 ms
Beats 71.28%

```

## 3) 要点解説

* **全表更新は I/O 支配**：式の違い（`CASE` / 配列）で速度差はほぼ出ません。可読性優先が吉。
* **無関係行のガード**：`WHERE sex IN ('m','f')` を付けると、将来 `NULL` や他値混在でも余計な書き換えを避けられます。
* **スキーマ設計の選択肢**：`ENUM('m','f')` でも、`TEXT + CHECK (sex IN ('m','f'))` でも今回の更新は同様に動作します。

## 4) 計算量（概算）

* 行数を `N` とすると **O(N)**（テーブルスキャン＋各行 1 回の更新判定）。
* 物理 I/O と WAL（書き込み前ログ）の負荷が支配的。巨大表ではバッチ分割でロック／レプリ遅延を緩和可能（総時間はやや増）。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Salary テーブル 全行] --> B[CASE で m と f を入れ替え]
  B --> C[列は id name sex salary のまま]
  C --> D[出力: sex が m/f 反転 他列は不変]
```

結論：**ほぼ頭打ち**です。
191–212ms の差は I/O と計測ゆらぎの範囲。すでに一番効く施策（`WHERE sex IN ('m','f')` で“必要行だけ更新”）を入れて 212→193ms に落ちています。これ以上は **物理 I/O と WAL** が支配的なので、式の書き方を変えても伸びません。

それでも実務で効く“次の一手”は以下。

## まだ詰められるポイント（PostgreSQL実運用）

1. **HOT Update を効かせる**（インデックス更新を回避）

   * `sex` に **インデックスを貼らない**（貼っているなら、トグル時は全件 index update が発生して遅くなる）。
   * ページ内に空きが必要なので **fillfactor を下げる（例: 90）**。将来の UPDATE でページ内移動＝HOT になりやすく、**WAL/インデックス更新が減少**。

     ```sql
     ALTER TABLE Salary SET (fillfactor = 90);
     -- 以後に挿入・更新されたページから効果。全面適用したければメンテ時間に REWRITE 系を計画。
     -- 例: VACUUM FULL や CLUSTER（重いので要メンテ枠）
     ```

2. **大規模表はバッチ更新**（ロック時間・レプリ遅延の平準化）
   総時間は微増しても可用性が上がります。`ctid` を使って素直に刻むのが PostgreSQL 流。

   ```sql
   -- 1回あたり 10k 行ずつ。0 行になったら終了。
   WITH chunk AS (
     SELECT ctid
     FROM Salary
     WHERE sex IN ('m','f')
     LIMIT 10000
   )
   UPDATE Salary s
   SET sex = CASE WHEN s.sex='m' THEN 'f' ELSE 'm' END
   FROM chunk
   WHERE s.ctid = chunk.ctid;
   ```

   *レプリケーション環境では遅延監視を。*

3. **無駄な書き込みをこれ以上減らす**
   すでに `WHERE sex IN ('m','f')` を付けていますが、将来 `NULL` や他値が混ざる懸念があるならガードはそのまま推奨。
   また、変更なし行を確実に避けるため `WHERE sex IN ('m','f')` は **必須**ではないものの、**実用最適**です（実測でも効いています）。

4. **WAL/チェックポイントまわりの運用**

   * ピーク外で実施、直前/直後に **手動 CHECKPOINT を避ける**（直撃すると I/O が重なる）。
   * レプリカがある場合はスロット有無や遅延を確認。
   * 耐障害要件が許す一括トグルなら、**単一トランザクションにまとめる**方がオーバーヘッドが小さくなりがち。

5. **スキーマ設計の将来最適化**（変更可の場合）
   性別を `ENUM('m','f')` / `TEXT` ではなく **`SMALLINT`/`BOOLEAN` + 表示層でマッピング**にすると、将来のトグルは

   ```sql
   UPDATE Salary SET sex = 1 - sex;  -- SMALLINT(0/1) の例
   ```

   で済み、データサイズも小さいため **WAL/I/O がさらに軽く**なります（ただしアプリ側の対応が必要）。

---

### どの書き方が最速？

`CASE`/`ARRAY` の 191–193ms は**互角**です。可読性で選んでください。
**“速さ”の残りしろは式ではなく**、HOT 可否・インデックス・fillfactor・バッチ運用・実行タイミングにあります。ビジネス的には、**インデックス設計の見直し＋バッチ更新**が費用対効果◎です。
